# USDA Data Loader - File Checklist

This script loads USDA actuarial data files into Snowflake.

## 📋 Instructions

1. **Initial Setup** (Run once):
   - Run STEP 0-5 to create database, network rules, and procedures
   
2. **Load Each File**:
   - Update configuration in STEP 1 with the file you want to load
   - Run STEP 6 to execute the load
   - Run STEP 7 to validate
   - Check off the file below

---

## 📁 Files to Load from `YYYY_ADM_YTD.zip`

| Status | Table Name | File Code | File Name |
|--------|-----------|-----------|-----------|
| ⬜ | `STATE_YTD` | `A00520` | `State_YTD` |
| ⬜ | `COUNTY_YTD` | `A00440` | `County_YTD` |
| ⬜ | `COMMODITY_YTD` | `A00420` | `Commodity_YTD` |
| ⬜ | `TYPE_YTD` | `A00540` | `Type_YTD` |
| ⬜ | `INSURANCEPLAN_YTD` | `A00460` | `InsurancePlan_YTD` |
| ⬜ | `INSURANCEOFFER_YTD` | `A00030` | `InsuranceOffer_YTD` |
| ⬜ | `COVERAGELEVELDIFFERENTIAL_YTD` | `A01040` | `CoverageLevelDifferential_YTD` |
| ⬜ | `SUBSIDYPERCENT_YTD` | `A0000` | `SubsidyPercent_YTD` |
| ⬜ | `PRICE_YTD` | `A00810` | `Price_YTD` |

**Mark files as complete:** Change ⬜ to ✅ when done!

---

## 🚀 Quick Reference - Configuration Examples

Copy and paste these into **STEP 1** of your SQL script:

### STATE File
```sql
SET FILE_CODE = 'A00520';
SET FILE_NAME = 'State_YTD';
SET TABLE_NAME = 'STATE';
```

### COUNTY File
```sql
SET FILE_CODE = 'A00440';
SET FILE_NAME = 'County_YTD';
SET TABLE_NAME = 'COUNTY';
```

### COMMODITY File
```sql
SET FILE_CODE = 'A00420';
SET FILE_NAME = 'Commodity_YTD';
SET TABLE_NAME = 'COMMODITY';
```

### TYPE File
```sql
SET FILE_CODE = 'A00540';
SET FILE_NAME = 'Type_YTD';
SET TABLE_NAME = 'TYPE';
```

### INSURANCEPLAN File
```sql
SET FILE_CODE = 'A00460';
SET FILE_NAME = 'InsurancePlan_YTD';
SET TABLE_NAME = 'INSURANCE_PLAN';
```

### INSURANCEOFFER File
```sql
SET FILE_CODE = 'A00030';
SET FILE_NAME = 'InsuranceOffer_YTD';
SET TABLE_NAME = 'INSURANCE_OFFER';
```

### COVERAGELEVELDIFFERENTIAL File
```sql
SET FILE_CODE = 'A00020';
SET FILE_NAME = 'CoverageLevelDifferential_YTD';
SET TABLE_NAME = 'COVERAGELEVELDIFFERENTIAL_YTD';
```

### SUBSIDYPERCENT File
```sql
SET FILE_CODE = 'A00060';
SET FILE_NAME = 'SubsidyPercent_YTD';
SET TABLE_NAME = 'SUBSIDYPERCENT_YTD';
```

### PRICE File
```sql
SET FILE_CODE = 'A00810';
SET FILE_NAME = 'Price_YTD';
SET TABLE_NAME = 'PRICE';
```

---

## 🔄 Workflow

1. Choose a file from the table above
2. Copy the configuration code for that file
3. Paste into STEP 1 of your SQL script
4. Run STEP 6 (Execute Load)
5. Run STEP 7 (Validation)
6. Mark the file as complete (✅)
7. Repeat for next file

---

## 💡 Example

**To load the STATE file:**

1. Update STEP 1:
```sql
SET FILE_CODE = 'A00004';
SET FILE_NAME = 'State_YTD';
SET TABLE_NAME = 'STATE_YTD';
```

2. Run STEP 6:
```sql
CALL ANALYTICS.PUBLIC.LOAD_USDA_FILE(...);
```

3. Run STEP 7 to verify

4. Mark complete:
| Status | Table Name | File Code | File Name |
|--------|-----------|-----------|-----------|
| ✅ | `STATE_YTD` | `A00004` | `State_YTD` |

---

## 📝 Notes

- All files come from the same ZIP: `2025_ADM_YTD.zip`
- Change `YEAR` variable to load different years
- Use `APPEND_MODE = 0` to overwrite, `1` to append
- The stored procedure automatically creates tables if they don't exist

In [ ]:
-- ====================================================================
-- STEP 0: Initial Database Setup (Fresh Instance)
-- ====================================================================

-- Use account admin role
USE ROLE ACCOUNTADMIN;

-- Create database if it doesn't exist
CREATE DATABASE IF NOT EXISTS CAPITAL_MARKETS_SANDBOX;

-- Create schema if it doesn't exist
CREATE SCHEMA IF NOT EXISTS CAPITAL_MARKETS_SANDBOX.PUBLIC;

-- Set context
USE DATABASE CAPITAL_MARKETS_SANDBOX;
USE SCHEMA PUBLIC;
USE WAREHOUSE COMPUTE_WH;

-- Verify setup
SELECT CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_ROLE(), CURRENT_WAREHOUSE();

In [ ]:
-- ====================================================================
-- CONFIGURATION - Define everything here
-- ====================================================================

-- Year for the data
SET YEAR = 2025;

-- File to load (code only, without year prefix)
SET FILE_CODE = 'A00470';
SET FILE_NAME = 'IntendedUse_YTD';

-- Table configuration
SET TABLE_NAME = 'INTENDEDUSE_YTD';

-- Append/Overwrite mode
SET APPEND_MODE = 0;  -- 0 = overwrite, 1 = append

-- Set context
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE COMPUTE_WH;
USE DATABASE CAPITAL_MARKETS_SANDBOX;
USE SCHEMA PUBLIC;

-- Display configuration
SELECT 
    $YEAR AS YEAR,
    $FILE_CODE AS FILE_CODE,
    $FILE_NAME AS FILE_NAME,
    $TABLE_NAME AS TABLE_NAME,
    $APPEND_MODE AS APPEND_MODE;

In [ ]:
-- ====================================================================
-- STEP 2: Drop Existing Table (comment out to skip)
-- ====================================================================

-- If you want to drop and recreate, uncomment this line:
--DROP TABLE IF EXISTS CAPITAL_MARKETS_SANDBOX.PUBLIC.STATE_YTD;

-- Or use the parameterized version:
-- EXECUTE IMMEDIATE 'DROP TABLE IF EXISTS CAPITAL_MARKETS_SANDBOX.PUBLIC.' || $TABLE_NAME;

In [ ]:
-- Allow access to USDA website
CREATE OR REPLACE NETWORK RULE usda_file_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('pubfs-rma.fpac.usda.gov:443');

In [ ]:
-- Create integration that uses the network rule (only needs to run once)
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION usda_file_integration
  ALLOWED_NETWORK_RULES = (usda_file_network_rule)
  ENABLED = TRUE;

In [ ]:
CREATE OR REPLACE PROCEDURE CAPITAL_MARKETS_SANDBOX.PUBLIC.LOAD_USDA_FILE(
    SOURCE_URL VARCHAR,
    ZIP_FILENAME VARCHAR,
    TARGET_FILE VARCHAR,
    TARGET_TABLE VARCHAR,
    APPEND_MODE INTEGER
)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.10'
PACKAGES = ('snowflake-snowpark-python', 'requests')
HANDLER = 'main'
EXTERNAL_ACCESS_INTEGRATIONS = (usda_file_integration)
AS
$$
import snowflake.snowpark as snowpark
from snowflake.snowpark.types import StringType, StructType, StructField
import requests
import zipfile
import io
import csv
import gc

def main(session, source_url, zip_filename, target_file, target_table, append_mode):
    try:
        full_url = source_url + "/" + zip_filename
        
        response = requests.get(full_url, stream=True, timeout=600)
        response.raise_for_status()
        
        zip_content = io.BytesIO()
        downloaded = 0
        
        for chunk in response.iter_content(chunk_size=262144):
            if chunk:
                zip_content.write(chunk)
                downloaded += len(chunk)
        
        downloaded_mb = downloaded / (1024 * 1024)
        zip_content.seek(0)
        
        with zipfile.ZipFile(zip_content) as zf:
            if target_file not in zf.namelist():
                return "ERROR: File not found in ZIP"
            
            with zf.open(target_file, 'r') as compressed_file:
                text_file = io.TextIOWrapper(compressed_file, encoding='utf-8', errors='replace')
                reader = csv.reader(text_file, delimiter='|')
                headers = next(reader)
                
                # Clean column names
                clean_cols = []
                for col in headers:
                    clean_col = str(col).strip().upper()
                    clean_col = clean_col.replace(' ', '_').replace('-', '_').replace('.', '_')
                    clean_col = clean_col.replace('/', '_').replace('\\', '_').replace('(', '').replace(')', '')
                    if len(clean_col) > 255:
                        clean_col = clean_col[:255]
                    clean_cols.append(clean_col)
                
                # KEY FIX: Create schema with all STRING types
                schema = StructType([StructField(col, StringType()) for col in clean_cols])
                
                table_exists = False
                try:
                    session.sql("SELECT COUNT(*) FROM " + target_table).collect()
                    table_exists = True
                except:
                    table_exists = False
                
                if table_exists and append_mode == 0:
                    session.sql("TRUNCATE TABLE " + target_table).collect()
                
                batch_size = 1000
                batch = []
                total_rows = 0
                batch_num = 0
                first_batch = True
                
                for row in reader:
                    # KEY FIX: Replace empty strings with None (NULL)
                    cleaned_row = [None if val == '' else val for val in row]
                    batch.append(cleaned_row)
                    
                    if len(batch) >= batch_size:
                        batch_num += 1
                        total_rows += len(batch)
                        
                        # Create DataFrame with explicit STRING schema
                        sp_df = session.create_dataframe(batch, schema=schema)
                        
                        if first_batch and not table_exists:
                            sp_df.write.mode("overwrite").save_as_table(target_table)
                            first_batch = False
                        else:
                            sp_df.write.mode("append").save_as_table(target_table)
                        
                        batch = []
                        del sp_df
                        
                        if batch_num % 10 == 0:
                            gc.collect()
                
                if batch:
                    batch_num += 1
                    total_rows += len(batch)
                    sp_df = session.create_dataframe(batch, schema=schema)
                    
                    if first_batch and not table_exists:
                        sp_df.write.mode("overwrite").save_as_table(target_table)
                    else:
                        sp_df.write.mode("append").save_as_table(target_table)
        
        zip_content.close()
        del zip_content
        gc.collect()
        
        final_count = session.sql("SELECT COUNT(*) FROM " + target_table).collect()[0][0]
        
        result = "SUCCESS! Downloaded: " + str(round(downloaded_mb, 2)) + " MB, "
        result += "Loaded: " + str(total_rows) + " rows in " + str(batch_num) + " batches. "
        result += "Final: " + str(final_count) + " rows."
        
        return result
        
    except Exception as e:
        import traceback
        return "ERROR: " + str(e) + " | " + traceback.format_exc()[:500]
$$;

In [ ]:
-- Your existing code - DON'T CHANGE ANYTHING!
SET FULL_FILE_NAME = $YEAR || '_' || $FILE_CODE || '_' || $FILE_NAME || '.txt';
SET FULL_ZIP_NAME = $YEAR || '_ADM_YTD.zip';
SET FULL_TABLE_NAME = 'CAPITAL_MARKETS_SANDBOX.PUBLIC.' || $TABLE_NAME;
SET BASE_URL = 'https://pubfs-rma.fpac.usda.gov/pub/References/actuarial_data_master/' || $YEAR;

SELECT
    $BASE_URL AS URL,
    $FULL_ZIP_NAME AS ZIP_FILE,
    $FULL_FILE_NAME AS TARGET_FILE,
    $FULL_TABLE_NAME AS TABLE_NAME,
    $APPEND_MODE AS MODE;

CALL CAPITAL_MARKETS_SANDBOX.PUBLIC.LOAD_USDA_FILE(
    $BASE_URL,
    $FULL_ZIP_NAME,
    $FULL_FILE_NAME,
    $FULL_TABLE_NAME,
    $APPEND_MODE
);

In [ ]:
-- ====================================================================
-- STEP 7: Validation
-- ====================================================================

-- Row count (using variable directly in query)
SELECT COUNT(*) as TOTAL_ROWS 
FROM IDENTIFIER($TABLE_NAME);

-- Table structure
DESCRIBE TABLE IDENTIFIER($TABLE_NAME);

-- Sample data
SELECT * 
FROM IDENTIFIER($TABLE_NAME) 
LIMIT 10;

-- Summary
SELECT 
    'Validation Complete' AS STATUS,
    $TABLE_NAME AS TABLE_LOADED,
    $YEAR AS YEAR_LOADED,
    $APPEND_MODE AS MODE_USED;

In [ ]:
SELECT * 
FROM CAPITAL_MARKETS_SANDBOX.PUBLIC.COVERAGLEVELDIFFERENTIAL_YTD 
LIMIT 10;